In [1]:
%matplotlib inline

import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pickle
from tools import ColorBinner, ColorHistogram, ImageRemover, get_hog_features, draw_windows, get_heatmap, draw_labeled_bboxes, y_limits, find_bounding_boxes
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.ndimage.measurements import label

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
clf = pickle.load(open("classifier.pkl", "rb"))


In [3]:
class ClipProcessor:
    def __init__(self):
        self.frame = 0
        self.dump_detections = True
        self.heatmaps = []
    
    def process_image(self, image):
        good_windows = []
        for window_size in [48, 64, 96, 128, 160]:
            good_windows.extend(find_bounding_boxes(image, clf, window_size, x_range=[400, None], y_range=y_limits[window_size]))

        heatmap = get_heatmap(image, good_windows)
        heatmap[heatmap < 0.1] = 0
        self.heatmaps.append(heatmap)
        if len(self.heatmaps) > 6:
            self.heatmaps.pop(0)

        sum_heatmap = self.heatmaps[0].copy()
        weight = 1
        for w, heatmap in enumerate(self.heatmaps[1:]):
            sum_heatmap = sum_heatmap + heatmap
            weight += 1

        sum_heatmap /= weight
        sum_heatmap[sum_heatmap < 0.1] = 0
            
        labels = label(sum_heatmap)
        draw_img = draw_labeled_bboxes(image, labels)

        cv2.imwrite(
            "frames_raw/{}.png".format(self.frame), 
            image[..., ::-1]
        )

        cv2.imwrite(
            "frames_boxed/{}.png".format(self.frame), 
            draw_img[..., ::-1]
        )
        
        self.frame += 1
        return draw_img

In [7]:
project_output_path = "project_video_output.mp4"

In [8]:
project_video = VideoFileClip("project_video.mp4")

processor = ClipProcessor()
project_video_output = project_video.fl_image(processor.process_image)

# %time 
project_video_output.write_videofile(project_output_path, audio=False)

[MoviePy] >>>> Building video test_video_output.mp4
[MoviePy] Writing video test_video_output.mp4


 97%|█████████▋| 38/39 [00:42<00:01,  1.10s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_output.mp4 



In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output_path))